In [1]:
# -*- coding:utf-8 -*-
import requests
import json
import time 
import pandas as pd
from bs4 import BeautifulSoup

from multiprocessing import Pool
from multiprocessing.dummy import Pool as ThreadPool
from requests.exceptions import RequestException
import re





In [2]:
def get_one_page(url):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            response.encoding = 'utf-8'
            return response.text
        return None
    except RequestException:
        print('error')
        return None

In [3]:
def parse_one_page(html):
    soup = BeautifulSoup(html)
    items = soup.find_all('li', class_= 'gl-item')
    #count = 0
    price_items = ''
    comment_items = ''
    id_items = []
    url_img_em_lists = []
    data_list=[]
    for item in items:
        ls = '<a href="//(.*?)" target="_blank">.*?<img data-img="1".*?"//(.*?)".*?<a.*?target="_blank".*?<em>(.*)</em>'
        ls_pattern = re.compile(ls, re.S)
        lists = re.findall(ls_pattern, str(item))
        for list in lists:
            # 获取链接，图片，文字描述
            url_img_em_lists += list
            
         

        for id_list in id_lists:
            comment_items += id_list + ','
            price_items += 'J_' + id_list + ','
        id_items += id_lists
    price_url = 'http://p.3.cn/prices/mgets?callback=jQuery9595587&ext=11000000&pin=&type=1&area=1_72_4137_0&skuIds=' \
                 + price_items + '&pdbp=0&pdtk=&pdpin=&pduid=15053986380611645853745&source=list_pc_front'      
    price_response = requests.get(price_url)
    price_response.encoding = 'utf-8'
    price_ls = '"p":"(.*?)"'
    price_pattern = re.compile(price_ls, re.S)
    # 获取价格
    price_lists = re.findall(price_pattern, price_response.text)
    
    comment_url = 'http://club.jd.com/comment/productCommentSummaries.action?my=pinglun&referenceIds=' + comment_items\
            + 'llback=jQuery3989310'
    #print(price_url)    
    #print(comment_url)   
    comment_response = requests.get(comment_url)
    comment_response.encoding = 'GBK'
    comment_ls = '"CommentCountStr":"(.*?)"'
    comment_pattern = re.compile(comment_ls, re.S)
    # 获取评论数目
    comment_lists = re.findall(comment_pattern, comment_response.text)
    
    
    for index in range(0,len(comment_lists)):
        data_list.append(url_img_em_lists[3*index])
        data_list.append(url_img_em_lists[3*index+1])
        data_list.append(url_img_em_lists[3*index+2].strip())
        data_list.append(price_lists[index])
        data_list.append(comment_lists[index])
        
        new_list=[data_list[i:i+5] for i in range(0,len(data_list),5)]
    return new_list        



In [4]:
def write_to_excel(content):
    global df_empty
    tmp_df = pd.DataFrame(content,columns=['链接','图片','描述','价格','评论数目'])
    tmp_df['描述'] = tmp_df['描述'].str.replace('<span.*?</span>\s*', '')
    df_empty = pd.concat([df_empty,tmp_df],axis=0)    
    #df_empty.to_csv('./jd_computer.csv',index=False,encoding='utf_8_sig')

In [7]:
def main(offset):
    url = 'http://list.jd.com/list.html?cat=670,671,672&page=' + str(offset) + '&sort=sort_totalsales15_desc&trans=1&JL=6_0_0#J_main'
    html = get_one_page(url)
    #write_to_excel(parse_one_page(html))
    return parse_one_page(html)



In [8]:
res=main(1)

C:\Users\pujing\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\Users\pujing\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


In [9]:
res

[['item.jd.com/5025518.html',
  'img13.360buyimg.com/n7/jfs/t7942/139/3840163775/188824/39be5a8d/59ffd41dN90ac91a2.jpg',
  '联想(Lenovo)拯救者R720 15.6英寸游戏笔记本电脑(i5-7300HQ 8G 1T+128G SSD GTX1050Ti 4G IPS 黑)',
  '6399.00',
  '6.6万+'],
 ['item.jd.com/5225346.html',
  'img11.360buyimg.com/n7/jfs/t5935/195/2108753717/176060/c849dcb6/593a49a3Nf9c2a052.jpg',
  'Apple MacBook Air 13.3英寸笔记本电脑 银色(2017新款Core i5 处理器/8GB内存/128GB闪存 MQD32CH/A)',
  '6988.00',
  '15万+'],
 ['item.jd.com/4163951.html',
  'img11.360buyimg.com/n7/jfs/t11746/357/1277342409/188824/39be5a8d/59ffd46eN750fc606.jpg',
  '联想(Lenovo)拯救者R720 15.6英寸游戏笔记本电脑(i7-7700HQ 8G 1T+128G SSD GTX1050Ti 4G IPS 黑)',
  '7399.00',
  '6.6万+'],
 ['item.jd.com/1378536.html',
  'img11.360buyimg.com/n7/jfs/t2128/108/878034220/94296/c0743107/5631d26bNdbc7ec32.jpg',
  'Apple MacBook Air 11.6英寸笔记本电脑 银色(Core i5 处理器/4GB内存/128GB SSD闪存 MJVM2CH/A)',
  '4999.00',
  '8.9万+'],
 ['item.jd.com/4824733.html',
  'img13.360buyimg.com/n7/jfs/t10234/128/2195912728/319946/b50a9

In [12]:
#df_empty = pd.DataFrame(columns=['链接','图片','描述','价格','评论数目'])  
df_empty = pd.DataFrame()
if __name__ == '__main__':
#     start_time = time.time() 
#     for i in range(1,11):
#         main(i)
    start_time = time.time() 
    pool = ThreadPool(5)
    pool.map(main, [i for i in range(1,1057)])
    pool.close()
    pool.join()
    end_time = time.time() 
    delta = end_time - start_time
    print(delta)
    print(df_empty.shape)
    
    df_empty.to_csv('./jd_computer.csv',index=False,encoding='utf_8_sig')
    print('ok')


E:\Anaconda3\lib\site-packages\bs4\__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


298.85409331321716
(63081, 5)
ok
